In [1]:
import requests
import re
import time
import json
import pandas as pd

In [246]:
import twitter
api = twitter.Api(consumer_key='UlwUFuQEQggWh144h2Tw2YGb9',
                      consumer_secret='nQDMPRol2e7ypItxPvG2wL4xkE4XZXrPBcwKvToU0IyiBDYiOF',
                      access_token_key='2377369042-V5IvDa8HH5K2gRLJ4tkYvPt3kXtC15xDYEFTpIV',
                      access_token_secret='3ZqbZ7NB0r57tNThW8xUKujgg0rxB1sjTUcxIvNZsfUPt')
def postMessage(message):
    status = api.PostUpdate(message, in_reply_to_status_id='1188011716220784640')

In [254]:
def alertNow(amount):
    '''alerts if the last 50 transactions passed the amount
    '''
    def computeDataFrame():
        def getData():
            '''Gets the transaction info for the last transactions.'''
            regex = r'''\/tx\/(?P<txnHash>0x[\da-z].+?)'.*?title='.*?'>(?P<date>[^<]+?)<.*?\/address\/(?P<from>0x[\da-z]+?)(?:#|').*?\/address\/(?P<to>0x[\da-z]+?)(?:#|').*?<td>(?P<value>[\d\.,]+?)<.*?\/token\/(?P<token>0x[\da-z]+?)'.*?>([^>]*?)\(([^\)<>]*?)\)'''
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            r = requests.get('https://etherscan.io/tokentxns', headers=headers)
            text = r.text
            res = re.findall(regex, text)
            data =  pd.DataFrame(res ,columns = ['hash','date', 'from','to','value','token', 'name', 'initials'])
            data.loc[:,'value'] = data.value.apply(lambda x: float(x.replace(',','')))
            return data
        data = getData()
        def queryPrice(tokenArray):
            ''' Gets the price of the tokens in the tokenArray'''
            def getPriceSmall(tokenArray):
                r = requests.get('https://min-api.cryptocompare.com/data/pricemulti?fsyms={}&tsyms=USD'.format(','.join(tokenArray)))
                text = r.text
                return text
            tokenArray = list(set(data.initials[:50]))
            tokenArrays = [tokenArray[i*30:(i+1)*30] for i in range(len(tokenArray)//30+1)]
            res = {}
            for array in tokenArrays:
                small_res = json.loads(getPriceSmall(array))
                res.update(small_res)
            res = {k: v['USD'] for k, v in res.items()}   
            return res
        prices = queryPrice(data.initials)  
        def getPrice(x):
            if x in prices.keys():
                return prices[x]
        data.loc[:,'AmountUSD']= data.initials.apply(getPrice) * data.value
        return data
    data = computeDataFrame()
    bigTrans = data.loc[data['AmountUSD']>=amount]
    if len(bigTrans):
        messages = []
        for i, row in bigTrans.iterrows():
                message = 'Big Transaction Detected ! at {}, from {} to {} with a value of {}{} ({}) which is around {:.0f} USD. #ERC20Elephant'.format(
                    row.date, row['from'] , row.to, row.value, row.name, row.initials, row.AmountUSD)  
                messages += [message]
        return messages
    return False

In [ ]:
import traceback

while True:
    try:
        messages = alertNow(1e5)
        if messages:
            print(messages)
            for mes in messages:
                postMessage(mes)
    except Exception as e:
        print(e)
        print(traceback.format_exc())
    time.sleep(30)

['Big Transaction Detected ! at 2019-10-26 13:56:37, from 0x6cc5f688a315f3dc28a7781717a9a798a59fda7b to 0xd7533c972f92189b651a8258124241f94bd8d122 with a value of 200000.025 (USDT) which is around 200800 USD. #ERC20Elephant', 'Big Transaction Detected ! at 2019-10-26 13:56:28, from 0xea5ee32f3a63c3fabb311c6e8c985d308a53dcc1 to 0xf5dce57282a584d2746faf1593d3121fcac444dc with a value of 231921.66294849 (DAI) which is around 238415 USD. #ERC20Elephant']
('Connection aborted.', error("(104, 'ECONNRESET')",))
Traceback (most recent call last):
  File "<ipython-input-256-be99eac69f3f>", line 9, in <module>
    postMessage(mes)
  File "<ipython-input-246-65655672fbcf>", line 7, in postMessage
    status = api.PostUpdate(message, in_reply_to_status_id='1188011716220784640')
  File "/home/tomas/anaconda3/lib/python2.7/site-packages/twitter/api.py", line 1176, in PostUpdate
    resp = self._RequestUrl(url, 'POST', data=parameters)
  File "/home/tomas/anaconda3/lib/python2.7/site-packages/twitter